Anushna Prakash  
DATA 512 - Human-Centered Data Science  
October 5, 2021  
# A1 - Data Curation

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [2]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

## Part I: Data Acquisition

In [3]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'

endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/{project}/{access}/{agent}/{granularity}/{start}/{end}'

In [4]:
# Use first of September to get complete data thru August 1, 2021
end_date = '20210901' + '00'

params_legacy_desktop = {
    "project" : "en.wikipedia.org",
    "access-site" : "desktop-site",
    "granularity" : "monthly",
    "start" : "2001010100",
    # for end use 1st day of month following final month of data
    "end" : end_date
}

params_legacy_mobile = {
    "project" : "en.wikipedia.org",
    "access-site" : "mobile-site",
    "granularity" : "monthly",
    "start" : "2001010100",
    # for end use 1st day of month following final month of data
    "end" : end_date
}

params_pageviews_desktop = {
    "project" : "en.wikipedia.org",
    "access" : "desktop",
    "agent" : "user",
    "granularity" : "monthly",
    "start" : "2001010100",
    # for end use 1st day of month following final month of data
    "end" : end_date
}

params_pageviews_app = {
    "project" : "en.wikipedia.org",
    "access" : "mobile-app",
    "agent" : "user",
    "granularity" : "monthly",
    "start" : "2001010100",
    # for end use 1st day of month following final month of data
    "end" : end_date
}

params_pageviews_mobileweb = {
    "project" : "en.wikipedia.org",
    "access" : "mobile-web",
    "agent" : "user",
    "granularity" : "monthly",
    "start" : "2001010100",
    # for end use 1st day of month following final month of data
    "end" : end_date
}

headers = {
    'User-Agent': 'https://github.com/anushnap',
    'From': 'anushnap@uw.edu'
}

In [5]:
def api_call(endpoint,parameters):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response

In [6]:
legacy_monthly_pageviews_desktop = api_call(endpoint_legacy, params_legacy_desktop)
legacy_monthly_pageviews_mobile = api_call(endpoint_legacy, params_legacy_mobile)
pageviews_monthly_pageviews_desktop = api_call(endpoint_pageviews, params_pageviews_desktop)
pageviews_monthly_pageviews_app = api_call(endpoint_pageviews, params_pageviews_app)
pageviews_monthly_pageviews_mobileweb = api_call(endpoint_pageviews, params_pageviews_mobileweb)

In [8]:
with open('./data_raw/legacy_desktop-site_200712-201607.json', 'w', encoding='utf-8') as f:
    json.dump(legacy_monthly_pageviews_desktop, f, ensure_ascii = False, indent=4)

with open('./data_raw/legacy_mobile-site_200712-201607.json', 'w', encoding='utf-8') as f:
    json.dump(legacy_monthly_pageviews_mobile, f, ensure_ascii = False, indent=4)

with open('./data_raw/pagecounts_desktop-site_200712-202108.json', 'w', encoding='utf-8') as f:
    json.dump(pageviews_monthly_pageviews_desktop, f, ensure_ascii = False, indent=4)

with open('./data_raw/pagecounts_mobile-web_200712-202108.json', 'w', encoding='utf-8') as f:
    json.dump(pageviews_monthly_pageviews_mobileweb, f, ensure_ascii = False, indent=4)

with open('./data_raw/pagecounts_mobile-app_200712-202108.json', 'w', encoding='utf-8') as f:
    json.dump(pageviews_monthly_pageviews_app, f, ensure_ascii = False, indent=4)

## Part 2: Data Processing

In [14]:
def clean_df(df):
    df['year'] = df['timestamp'].str.slice(start = 0, stop = 4).astype(int)
    df['month'] = df['timestamp'].str.slice(start = 4, stop = 6).astype(int)
    return df[['year', 'month', 'count']]

df_desktop = clean_df(pd.json_normalize(legacy_monthly_pageviews_desktop['items'])) 
# df_desktop = clean_df(df_desktop)
print(df_desktop.info())
df_mobile = clean_df(pd.json_normalize(legacy_monthly_pageviews_mobile['items']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   year    105 non-null    int64
 1   month   105 non-null    int64
 2   count   105 non-null    int64
dtypes: int64(3)
memory usage: 2.6 KB
None


In [21]:
df = df_desktop.merge(df_mobile, how = 'left', on = ['year', 'month']).rename(columns = {'count_x': 'pagecounts_desktop_views', 'count_y': 'pagecounts_mobile_views'})

In [25]:
df.fillna(0, inplace = True)
df

,year,month,pagecounts_desktop_views,pagecounts_mobile_views
0,2007,12,2998331524,0.000000e+00
1,2008,1,4930902570,0.000000e+00
2,2008,2,4818393763,0.000000e+00
3,2008,3,4955405809,0.000000e+00
4,2008,4,5159162183,0.000000e+00
...,...,...,...,...
100,2016,4,5572235399,3.301385e+09
101,2016,5,5330532334,3.418436e+09
102,2016,6,4975092447,3.372618e+09
103,2016,7,5363966439,3.500661e+09
